### Attempt 1: Chignolin, automatic assembly

In [41]:
import MDAnalysis as mda
import MDAnalysis.converters.RDKitParser as RDKitParser
import rdkit.Chem as Chem

In [40]:
pdb_file = '/Users/bendupontjr/mphil_files/all_mphil_code/pyclops/examples/pdbs/chignolin.pdb'
mol = Chem.MolFromPDBFile(pdb_file, sanitize=True, removeHs=True)

u = RDKitParser.RDKitParser(pdb_file)

#mol.GetAtomWithIdx(5).GetPDBResidueInfo().GetResidueName()

mol.GetAtomWithIdx(12).GetPDBResidueInfo().GetName().strip()

'N'

In [43]:
len(mol.GetAtoms())

93